In [2]:
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor

import torchvision
import torch
import torchvision.transforms as transform

In [3]:
batch_size = 50

path = './datasets'

trainset = CIFAR10(root=path, train=True,
                                        download=True, transform=ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2, drop_last=True)

testset = CIFAR10(root=path, train=False,
                                       download=True, transform=ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2, drop_last=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
from torch import nn

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # 32, 32, 3
        self.proc1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 16, 16, 64
        self.proc2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(), 
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # 8, 8, 128
        self.proc3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2)
        )
        # 4, 4, 256
        
        self.classfier = nn.Sequential(
            nn.Linear(4*4*256, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 10)
        )       
        
    
    def forward(self, x):
        
        x = self.proc1(x)
        # print(x.size())
        
        x = self.proc2(x)
        # print(x.size())

        x = self.proc3(x)
        # print(x.size())
        
        x = x.view(-1, 4*4*256)

        x = self.classfier(x)
        return x



In [5]:
# import time as t
# import numpy as np
# ########### Write Your Code Here ###########
# LEARNING_RATE = 0.01
# device = torch.device('mps')
# model = CNN().to(device)
# loss_fn = nn.CrossEntropyLoss().to(device)
# opt = torch.optim.RMSprop(model.parameters(), lr=LEARNING_RATE)
# ############################################

# res = []

# Epochs = 500
# for i in range(Epochs):
#     last_time = t.time()
#     for cnt, (x, y) in enumerate(trainloader):
#         samples = x.to(device)
#         print(samples.size())
#         y = y.to(device)
#         output = model(samples)
        
        
#         loss = loss_fn(output, y.to(device))
#         opt.zero_grad()
        
#         loss.backward()
#         opt.step()
#         y_pred = torch.argmax(output, dim=1).to(device)        
#         acc = (torch.argwhere(y_pred == y)).size(0)/y.size(0)
#         res.append(acc)
        
    
#     mean_acc = np.mean(res)
#     res.clear()
#     this_time = t.time()
#     print("Epoch:{}/{}, step:{}, loss:{:.4f}, time:{:.2f}".format(i + 1, Epochs, i + 1, loss.item(), this_time - last_time))
#     print("acc:{:.4f}".format(mean_acc))
    
    
#     if((i + 1)%(Epochs/(Epochs/10)) == 0):
#         torch.save(model.state_dict(), 'model' + str(model)[:3] + "_" + str(opt)[:7] + "lr=" + str(LEARNING_RATE) + "_" + str(i/(Epochs/100) + 1) + '.pth')
#         print(str(opt)[:7])
#         print("Checkpoint save!")

        


In [21]:
from sklearn.metrics import classification_report
import numpy as np

model = CNN()

finalModel = "model_ada"

model.load_state_dict(torch.load(finalModel + ".pth", map_location=torch.device("mps")))

model.eval()

CNN(
  (proc1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (proc2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (proc3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classfier): Sequential(
    (0): Linear(in_features=4096, out_features=512, bias=True)
    (1): ReLU()
    (2): Dro

In [22]:
acc = []

y_pred = torch.tensor([], dtype=torch.int32)
y_test = torch.tensor([], dtype=torch.int32)

for data in testloader:
    x, y = data
    tmp = torch.argmax(model(x), dim=1)
    y_pred = torch.cat((y_pred, tmp), dim=0)
    y_test = torch.cat((y_test, y), dim=0)

In [23]:
from sklearn.metrics import classification_report

arr_pred = np.array(y_pred)
arr_test = np.array(y_test)

print(classification_report(arr_test, arr_pred))

              precision    recall  f1-score   support

           0       0.75      0.78      0.76      1000
           1       0.86      0.81      0.83      1000
           2       0.66      0.57      0.62      1000
           3       0.53      0.54      0.54      1000
           4       0.67      0.66      0.67      1000
           5       0.62      0.63      0.62      1000
           6       0.77      0.79      0.78      1000
           7       0.74      0.76      0.75      1000
           8       0.83      0.82      0.82      1000
           9       0.76      0.83      0.79      1000

    accuracy                           0.72     10000
   macro avg       0.72      0.72      0.72     10000
weighted avg       0.72      0.72      0.72     10000

